In [ ]:
conda install tensorflow

In [ ]:
conda install keras

In [8]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report as cr
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib

### Load dataset and split them into train and test set

In [9]:
path = 'C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/data/train_set/'
data_unsplit = pd.read_csv(path + "data_unsplit.csv")
X_dis=data_unsplit.iloc[:,1:-1]
y_dis=data_unsplit["emotion_idx"]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_dis = scaler.fit_transform(X_dis)

train_x_dis,test_x_dis,train_y_dis,test_y_dis=train_test_split(X_dis,y_dis,test_size=0.2,random_state=3662)

### Use K-Fold to estimate the accuracy (49.36%)

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(??????????, X_dis,y_dis, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

### Fit the final model and save it

In [ ]:
t0 = time.time()
MLP_model_final.fit(X_dis,y_dis)
print("done in %0.3fs" % (time.time() - t0))

In [ ]:
joblib.dump(logistic_model_final,'MLP_model_final.m')

In [ ]:
#joblib.load('C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/doc/MLP_model_final.m')

In [10]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost
from sklearn.neural_network import MLPClassifier

In [11]:
gbm_baseline= GradientBoostingClassifier(n_estimators=100  , max_depth= 1,learning_rate=0.1)

In [12]:
gbm_improved= GradientBoostingClassifier(n_estimators=100  , max_depth= 2,learning_rate=0.1)

In [13]:
logistic_model_final =LogisticRegression(C=0.01,
                                         dual=False,
                                         fit_intercept=True,
                                         intercept_scaling=1,
                                         max_iter=300,
                                         multi_class='multinomial',
                                         penalty='l2',
                                         solver='newton-cg',
                                         tol=0.0001)

In [14]:
MLP_model_final =MLPClassifier(early_stopping=True,
                              hidden_layer_sizes=(3000,),
                              learning_rate='adaptive',
                              solver='lbfgs',
                              validation_fraction=0.2)

In [15]:
svm_model_final =SVC(C=0.1,decision_function_shape='ovr',degree=2,gamma=0.1,kernel='linear')

In [16]:
xgboost_model_final =xgboost.XGBClassifier(max_depth=4,n_estimators=50,learning_rate=0.1,
                       min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,reg_alpha=0.005)

In [17]:
voting_clf = VotingClassifier(
    estimators=[('gbm_clf',gbm_improved),
                ('log_clf', logistic_model_final),
                ('svm_clf', svm_model_final),
                ('xgb_clf', xgboost_model_final),
                ('MLP_clf',MLP_model_final)],
    voting='hard',
    weights=[1,1,1,1,1],
    n_jobs=8
)

# voting_clf.fit(train_x_dis, train_y_dis)
# voting_clf.score(test_x_dis, test_y_dis)

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(voting_clf, X_dis,y_dis, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [22]:
voting_clf.score(test_x_dis, test_y_dis)

0.502

In [3]:
from sklearn.ensemble import RandomForestClassifier
import time

In [ ]:
t0 = time.time()
voting_clf.fit(X_dis,y_dis)
print("done in %0.3fs" % (time.time() - t0))

In [ ]:
t0 = time.time()
voting_clf.predict(X_dis)
print("done in %0.3fs" % (time.time() - t0))

In [ ]:
t0 = time.time()
voting_clf.score(X_dis,y_dis)
print("done in %0.3fs" % (time.time() - t0))